In [95]:
## Demultiplexing code copied from laura

In [96]:

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import gzip
from sklearn.neighbors import KernelDensity
from scipy.signal import argrelextrema
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

In [97]:
def read_cell_whitelist(barcode_file):
    whitelist = []
    with open(barcode_file) as wl:
        whitelist = [x[:-2] for x in wl.read().splitlines()]
    return whitelist

In [98]:
def ms_whitelist(barcode_file, sep = ',', skipfirst = False):
    if not skipfirst:
        return [x.split(sep)[0] for x in open(barcode_file).read().splitlines()]
    else:
         return [x.split(sep)[0] for x in open(barcode_file).read().splitlines()][1:]

In [99]:
def cells_to_reads(read1, read2, white, multiseqBool = True):
    cells = {}
    with gzip.open(read1, 'rt') as r1File, gzip.open(read2, 'rt') as r2File:
        count = 0
        for i, ((title1, seq1, qual1), (title2, seq2, qual2)) in tqdm(enumerate(zip(FastqGeneralIterator(r1File), FastqGeneralIterator(r2File)))):
            # Get the pieces of interest
            cell = seq1[0:16]
            if multiseqBool:
                UMI = seq1[17:27]
                multiseq = seq2[0:8]
            else:
                UMI = seq1[17:29]
                multiseq = seq2[0:18]
            coding = seq2[38:]
            
            read = {}
            read['umi'] = UMI
            read['multiseq'] = multiseq
            read['coding'] = coding
            
            # Only add cell barcodes that are in the whitelist
            if cell in white:
                if cell not in cells:
                    cells[cell] = []

                cells[cell].append(read)
    return cells

In [100]:
def flatten_UMI(cells):
    newCells = {}
    for cellid, cell in tqdm(cells.items()): # for each cell (by cell barcode)
        umis = []
        firstUniqueReads = []
        for read in cell: # for each read with that cell barcode, keep only unique
            if read['umi'] not in umis: 
                firstUniqueReads.append(read)
                umis.append(read['umi'])
        newCells[cellid] = firstUniqueReads
    return newCells

In [101]:
def map_barcodes(cells, mswhite):
    cellBarcodes = {}
    for cellid, cell in tqdm(cells.items()):
        ms = {} # multiseq barcodes
        for read in cell:
            if read['multiseq'] in mswhite:
                if read['multiseq'] in ms:
                    ms[read['multiseq']] = ms[read['multiseq']] + 1
                else:
                    ms[read['multiseq']] = 1
        cellBarcodes[cellid] = ms
    return cellBarcodes

In [102]:
def plot_counts(cellBarcodes, name, save = False):
    
    all_list = [val for sublist in [list(x.keys()) for x in cellBarcodes.values()] for val in sublist]
    targets = sorted(set(all_list), key = lambda x: sum([x == y for y in all_list]), reverse = True)[:3]
    
    f, ax = plt.subplots(figsize=(5, 5))

    for target in targets:
        counts = []
        for cell in cellBarcodes:
            barcodes = cellBarcodes[cell]
            for barcode in barcodes:
                if barcode == target:
                    counts.append(barcodes[barcode])

        df = pd.DataFrame({'counts':counts})            
        x = df.counts.tolist()
        x = [np.log(i) for i in x]

        sns.kdeplot(x, label = target)
    if save:
        plt.savefig("results/" + name + '_counts.png', bbox_inches='tight')

In [103]:
def threshold_for_barcode(target, data, cache):
    # Get the counts for target barcode from all the cells
    if target in cache.keys():
        return cache[target]
        
    counts = []
    for cell in data:
        barcodes = data[cell]
        for barcode in barcodes:
            if barcode == target:
                counts.append(barcodes[barcode])

    x = [[np.log(i)] for i in counts]
    
    # instantiate and fit the KDE model
    kde = KernelDensity(bandwidth=.5, kernel='gaussian')
    kde.fit(x)
    x_d = np.linspace(-4,8,2000)
    logprob = kde.score_samples(x_d[:, None])
    minm = argrelextrema(np.exp(logprob), np.less)  # (array([2, 5, 7]),)
    if len(minm[0]) > 0:
        cache[target] = x_d[minm[0][0]]
        return x_d[minm[0][0]]
    else:
        return 100

In [104]:
def get_final_labels(cellBarcodes):
    cache = {}
    finalMSLabels = {}
    for cellid, barcodes in tqdm(cellBarcodes.items()):
        aboveThresh = []
        sorted_counts = sorted(barcodes.values(), reverse = True)
        quality = float("inf")
        if len(sorted_counts) > 1:
            quality = sorted_counts[0]/sorted_counts[1]
        for barcode, count in barcodes.items():
            count = barcodes[barcode]
            if count >= np.exp(threshold_for_barcode(barcode, cellBarcodes, cache)):
                aboveThresh.append(barcode)
        if len(aboveThresh) == 1:
            finalMSLabels[cellid] = (aboveThresh[0], quality)
        else:
            finalMSLabels[cellid] = ("DOUBLET", quality)
    return finalMSLabels

In [105]:
def write_csv(cellBarcodes, name): 
    df = pd.DataFrame([(cell, ms, q) for cell, (ms, q) in cellBarcodes.items()], columns = ['cell barcode', 'multiseq barcode', 'quality'])
    df.to_csv("results/" + name + "_barcode_identities.csv")

In [106]:
def run_all(read1, read2, cell_wl, ms_wl, label, multiseqBool = True, save = True):
    cells = cells_to_reads(read1, read2, cell_whitelist(cell_wl), multiseqBool)
    unique_reads = flatten_UMI(cells)
    if multiseqBool:
        cell_barcodes = map_barcodes(unique_reads, ms_whitelist(ms_wl))
    else:
        cell_barcodes = map_barcodes(unique_reads, ms_whitelist(ms_wl, sep = "\t", skipfirst = True))
    plot_counts(cell_barcodes, label, save)
    final_labels = get_final_labels(cell_barcodes)
    write_csv(final_labels, label)

In [107]:
cell_wl = '/wynton/scratch/bp205/raw/BC_MULTIseq_JW/737K-whitelist.txt'
ms_wl = '/wynton/scratch/bp205/raw/BC_MULTIseq_JW/LMOlist.csv'
label='PDX1'
multiseqBool=True

In [108]:
read1 = '/wynton/scratch/bp205/raw/BC_MULTIseq_JW/PDX1/PDX1_BAR/GCCAAT_S10_L004_R1_001.fastq.gz'
read2 = '/wynton/scratch/bp205/raw/BC_MULTIseq_JW/PDX1/PDX1_BAR/GCCAAT_S10_L004_R2_001.fastq.gz'

In [109]:
cell_whitelist = np.unique(read_cell_whitelist(cell_wl))

In [ ]:
cells = cells_to_reads(read1,read2,cell_whitelist,multiseqBool)

In [93]:
## Custom demultiplexing 
UMIs = []
cell_ids = pd.read_csv('/wynton/scratch/bp205/raw/BC_MULTIseq_JW/cell_ids.csv')
cellBC = np.unique(cell_ids['cellBC'].values.astype(np.unicode))
cells = {k:defaultdict(int) for k in cellBC}

with gzip.open(read1, 'rt') as r1File, gzip.open(read2, 'rt') as r2File:
    for i, ((title1, seq1, qual1), (title2, seq2, qual2)) in tqdm(enumerate(zip(FastqGeneralIterator(r1File), FastqGeneralIterator(r2File)))):        
        cell = seq1[0:16]
        if cell in cellBC:
            UMI = seq1[17:29]
            if UMI not in UMIs:
                UMIs.append(UMI)
                cells[cell][seq2[0:8]]+=1
        
        
        
#             # Get the pieces of interest
#             cell = seq1[0:16]
#             if multiseqBool:
#                 UMI = seq1[17:27]
#                 multiseq = seq2[0:8]
#             else:
#                 UMI = seq1[17:29]
#                 multiseq = seq2[0:18]
#             coding = seq2[38:]
            
#             read = {}
#             read['umi'] = UMI
#             read['multiseq'] = multiseq
#             read['coding'] = coding
            
#             # Only add cell barcodes that are in the whitelist
#             if cell in white:
#                 if cell not in cells:
#                     cells[cell] = []

#                 cells[cell].append(read)




KeyboardInterrupt: 

In [91]:
[k for k,x in cells.items() if len(x) > 0]

['GAAATGAGTCGCCATG']

In [92]:
cells['GAAATGAGTCGCCATG']

defaultdict(int,
            {'CCAGANTG': 2,
             'CCACANTG': 55,
             'GCAGGNAC': 9,
             'ATAGGTGC': 82,
             'GCAGTATC': 24241,
             'ATACGTGC': 17066,
             'GGAGAAGA': 13625,
             'GCATTATG': 3,
             'CCAGGTCT': 10,
             'CGAGGNCT': 3,
             'GCACTNTC': 1,
             'GTACCNCT': 1,
             'GCAGTNTC': 31,
             'ATAGTNCG': 1,
             'GCTAGNAC': 1,
             'TCACANCA': 13,
             'ATACGNGC': 27,
             'TTAGCNTG': 1,
             'TGAGANCT': 10,
             'AGATTNCG': 12,
             'CGAGANTC': 7,
             'GGAGANGA': 23,
             'TGAGANCA': 2,
             'AGAGANAT': 1,
             'GGAGANAA': 1,
             'GCATTNTC': 3,
             'GCACANGC': 22,
             'GCATGNAC': 17,
             'CTATGNCT': 1,
             'GGATANCT': 1,
             'TGCGANCT': 2,
             'ATAGTNCA': 1,
             'TTCACNTC': 1,
             'ATAGGNGC': 2,
        

In [73]:
d = defaultdict(int)

In [77]:
d['potadto']+=100

In [78]:
d

defaultdict(int, {'potato': 100, 'potadto': 100})

In [ ]:
def cells_to_reads(read1, read2, white, multiseqBool = True):
    cells = {}
    with gzip.open(read1, 'rt') as r1File, gzip.open(read2, 'rt') as r2File:
        count = 0
        for i, ((title1, seq1, qual1), (title2, seq2, qual2)) in tqdm(enumerate(zip(FastqGeneralIterator(r1File), FastqGeneralIterator(r2File)))):
            # Get the pieces of interest
            cell = seq1[0:16]
            if multiseqBool:
                UMI = seq1[17:27]
                multiseq = seq2[0:8]
            else:
                UMI = seq1[17:29]
                multiseq = seq2[0:18]
            coding = seq2[38:]
            
            read = {}
            read['umi'] = UMI
            read['multiseq'] = multiseq
            read['coding'] = coding
            
            # Only add cell barcodes that are in the whitelist
            if cell in white:
                if cell not in cells:
                    cells[cell] = []

                cells[cell].append(read)
    return cells